# Tool Binding

- Tool binding is the process where you register tools with a LLM so that :
    - The LLM knows what tools are available.
    - It knows what each tool does (via description).
    - It knowa what input format to use (via schema).

In [34]:
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, ToolMessage, AIMessage

model = init_chat_model(
    model="functiongemma:latest",
    model_provider="ollama",
    temperature = 0.0
)

messages = []

In [35]:
# Tool Creation
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [20]:
# Tool Binding
llm_withTools = model.bind_tools([multiply])
llm_withTools.invoke('hi how are you')

AIMessage(content='I am doing well, thank you for asking! I hope you are too. How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'functiongemma:latest', 'created_at': '2026-02-21T10:00:56.7419709Z', 'done': True, 'done_reason': 'stop', 'total_duration': 518204300, 'load_duration': 245617700, 'prompt_eval_count': 98, 'prompt_eval_duration': 16447800, 'eval_count': 24, 'eval_duration': 240865900, 'logprobs': None, 'model_name': 'functiongemma:latest', 'model_provider': 'ollama'}, id='lc_run--019c7fa5-289e-7d82-b678-a6cc8de7e2d4-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 98, 'output_tokens': 24, 'total_tokens': 122})

# Tool Calling

- Tool calling is the process where the LLM decides, during a conversation or task, that it needs to use a specific tool(function) - and generates a structured output with :
    - name of the tool
    - the arguments to call with it

- The LLM does not actually run the tool- It just suggests the tool and argument. The actual execution is handled by Langchain for you.

In [41]:
query = HumanMessage('can you multiply 4 and 3')
messages.append(query)

response = llm_withTools.invoke(messages)
messages.append(response)
for key in response:
  print(key)

('content', 'The product of 4 and 3 is 6.')
('additional_kwargs', {})
('response_metadata', {'model': 'functiongemma:latest', 'created_at': '2026-02-21T10:20:42.070878Z', 'done': True, 'done_reason': 'stop', 'total_duration': 634840600, 'load_duration': 322857000, 'prompt_eval_count': 152, 'prompt_eval_duration': 20450100, 'eval_count': 27, 'eval_duration': 260161500, 'logprobs': None, 'model_name': 'functiongemma:latest', 'model_provider': 'ollama'})
('type', 'ai')
('name', None)
('id', 'lc_run--019c7fb7-3e5a-72b1-98d0-713ddb6b13fb-0')
('tool_calls', [{'name': 'multiply', 'args': {'a': 4, 'b': 3}, 'id': '0dbd7d63-3f24-49f9-8c11-42e1d7ffad21', 'type': 'tool_call'}])
('invalid_tool_calls', [])
('usage_metadata', {'input_tokens': 152, 'output_tokens': 27, 'total_tokens': 179})


# Tool Execution

- Tool execution is the step where the actual python function(tool) is run using i/p args that LLM suggested during tool calling

In [42]:
if response.tool_calls:
  tool_msg = response.tool_calls[0]
  print(f'Calling: {tool_msg['name']} - ({tool_msg['args']})')

  result = multiply.invoke(tool_msg['args'])
  print(f'Result: {result}')

Calling: multiply - ({'a': 4, 'b': 3})
Result: 12


In [43]:
if response.tool_calls:
  tool_msg = response.tool_calls[0]
  print(f'Calling: {tool_msg['name']} - ({tool_msg['args']})')
  result = multiply.invoke(tool_msg)
  messages.append(result)
  print(result)
  print(type(result))
  

Calling: multiply - ({'a': 4, 'b': 3})
content='12' name='multiply' tool_call_id='0dbd7d63-3f24-49f9-8c11-42e1d7ffad21'
<class 'langchain_core.messages.tool.ToolMessage'>


In [44]:
final_result = llm_withTools.invoke(messages)
final_result

AIMessage(content='The product of 4 and 3 is 12.', additional_kwargs={}, response_metadata={'model': 'functiongemma:latest', 'created_at': '2026-02-21T10:20:42.9167047Z', 'done': True, 'done_reason': 'stop', 'total_duration': 488220300, 'load_duration': 328132300, 'prompt_eval_count': 189, 'prompt_eval_duration': 18085700, 'eval_count': 14, 'eval_duration': 131505100, 'logprobs': None, 'model_name': 'functiongemma:latest', 'model_provider': 'ollama'}, id='lc_run--019c7fb7-423b-7003-afdf-10ff8166e611-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 189, 'output_tokens': 14, 'total_tokens': 203})